# ~~~

## Эта тетрадка посвящена замене слов в предложениях из существующих реально разметок

Сюда надо перенести историю про одушевленность и переходность из первого файла по генерации.

И сюда же надо будет грузить наречия трех видов, которые сделала Вика вот в этом файле: 
https://docs.google.com/spreadsheets/d/1nxMI0hznhXCROKxScsugj6-viQ8ibCBWLsgANdT_cmQ/edit#gid=0 

In [ ]:
import os
import re
import json
from collections import defaultdict
import pympi
import pandas as pd
import numpy as np
from pprint import pprint

Сначала вспомогательный код для прочтения файлов с разметками:

In [ ]:
class DataToList:
    # TODO: support files with two signers
    def __init__(self, full_filename):
        self.eaf = pympi.Elan.Eaf(full_filename)
        if len(self.eaf.get_tier_names()) > 5:
            self.second_person = True
        else:
            self.second_person = False

    def read_data_rus(self):
        eaf_form = self.eaf.get_annotation_data_for_tier("Перевод")
        rus_sentences = []

        if eaf_form:
            for sign in eaf_form:
                # [0] is start time of annotation, [1] is end time
                russian = (sign[0], sign[1], sign[2])
                rus_sentences.append(russian)

        # print(rus_sentences)
        return rus_sentences

    def read_data_rsl(self):
        rus = self.read_data_rus()
        rsl_sentences_raw = []
        for sentence_rus in rus:
            sentence = []
            eaf_form_gloss = self.eaf.get_annotation_data_between_times("ПР-глосс",
                                                                   sentence_rus[0] - 2,
                                                                   sentence_rus[1] + 2)
            if eaf_form_gloss:
                # 2 - не время, а текст аннотации
                for word in eaf_form_gloss:
                    new_word = word[2].lower()

                    # проверить на дактиль и заменить его на <UNK> токен
                    #if re.match(r'.(-.)+', new_word):
                    #    sentence.append('<unk>')
                    #    continue

                    # проверить есть ли согласование
                    ending = ''
                    if ':' in new_word:
                        if new_word[1:].lower().startswith('ps'):
                            sentence.append(new_word[:4])
                            new_word = new_word[4:]
                        if new_word.lower().endswith('pl'):
                            ending = ':pl'
                            new_word = new_word[:-3]
                        if new_word.lower().endswith('ps'):
                            ending = new_word[-4:]
                            new_word = new_word[:-4]
                        sentence.append(new_word)
                        if ending != '':
                            sentence.append(ending)
                    else:
                        sentence.append(new_word)
            else:
                print("ERROR ERROR для предложения нет глосс ERROR ERROR")
            rsl_sentences_raw.append(sentence)

        # этот разделитель токенов - костыль, чтобы можно было использовать токенизацию
        # через пайплайн Field > TabularDataset > Iterator
        # То есть РЖЯ уже токенизирован, но это придется сделать еще раз
        rsl_sentences_raw = ['%>%'.join(s) for s in rsl_sentences_raw]
        # print(rsl_sentences_raw)
        return rsl_sentences_raw


def read_elan_files(elan_file_path):
    # Read elan files into two lists of sentences
    rus_sentences, rsl_sentences = [], []
    for dir_path, dir_names, filenames in os.walk(elan_file_path):
        print(filenames)
        for filename in filenames:
            if filename.endswith('.eaf'):
                full_filename = os.path.join(dir_path, filename)
                rus = [sentence[2] for sentence in DataToList(full_filename).read_data_rus()]
                rsl = DataToList(full_filename).read_data_rsl()
                rus_sentences.extend(rus)
                rsl_sentences.extend(rsl)
    return rus_sentences, rsl_sentences



In [ ]:
rus, rsl = read_elan_files("../translation/Разметки")

In [ ]:
print(len(rus), len(rsl))

Детур: сделаю txt файл из русских разметок, чтобы добавить их к нграммам.

In [ ]:
with open('real_rus_translations.txt', 'w', encoding='utf-8') as f:
    for sent in rus:
        f.write(sent + '\n')

Детур закончился

In [ ]:
split_rsl = [el.split("%>%") for el in rsl]

In [ ]:
pprint(split_rsl[:2])

## Замена наречий

Теперь попробоуем искать в предложениях на ржя наречия определенного типа и заменять их на наречия того же типа.

In [ ]:
time_adv = pd.read_csv("time_adverbs.csv", encoding="utf-8")

In [ ]:
time_adv.head()

In [ ]:
for adverb in time_adv.text:
    print(adverb.upper())
    for k, sentence in enumerate(split_rsl):
        if adverb in sentence:
            print(sentence)

### Теперь можно генерировать фейковые данные

In [ ]:
with open("time_adverbs_fake_data.tsv", "w", encoding="utf-8") as f:
    for adverb in time_adv.text:
        other = list(time_adv.text)
        other.remove(adverb)
        for k, sentence in enumerate(split_rsl):
            if adverb in sentence:
                for other_adverb in other:
                    rus_new = re.sub(adverb, other_adverb, rus[k], flags=re.I)
                    f.write(" ".join([other_adverb if word == adverb else word for word in sentence]) + "\t" +\
                            rus_new.capitalize() + "\n")

Перейдем к наречиям места

In [ ]:
place_adv = pd.read_csv("place_adverbs.csv", encoding="utf-8")

In [ ]:
place_adv.head()

In [ ]:
for adverb in place_adv.text:
    print(adverb.upper())
    for k, sentence in enumerate(split_rsl):
        if adverb in sentence:
            print(sentence)

In [ ]:
with open("place_adverbs_fake_data.tsv", "w", encoding="utf-8") as f:
    for adverb in place_adv.text:
        other = list(place_adv.text)
        other.remove(adverb)
        for k, sentence in enumerate(split_rsl):
            if adverb in sentence:
                for other_adverb in other:
                    rus_new = re.sub(adverb, other_adverb, rus[k], flags=re.I)
                    f.write(" ".join([other_adverb if word == adverb else word for word in sentence]) + "\t" +\
                            rus_new.capitalize() + "\n")

С наречиями места, если честно, получилась какая-то хрень.

Я отсмотрела глазами этот файл и поняла, что это вроде как не наречия, а скорее предлоги, потому что они постоянно задают какое-то композициональное значение... Даже типа не попадают обычно в русский перевод. Если я права, то это могло бы быть очень интересно с лингвистической точки зрения, поэтому я соберу реальные предложения с этими предлогами в один файлик, чтобы потом вместе его посмотреть.

In [ ]:
with open("place_adverbs_real_data.tsv", "w", encoding="utf-8") as f:
    for adverb in place_adv.text:
        f.write(adverb.upper() + "\n")
        for k, sentence in enumerate(split_rsl):
            if adverb in sentence:
                f.write(" ".join(sentence) + "\t" + rus[k].capitalize() + "\n")

И наконец наречия образа действия

In [ ]:
move_adv = pd.read_csv("move_adv.csv", encoding="utf-8")
move_adv.head()

In [ ]:
for adverb in move_adv.text:
    print(adverb.upper())
    for k, sentence in enumerate(split_rsl):
        if adverb in sentence:
            print(sentence)

In [ ]:
with open("move_adverbs_fake_data.tsv", "w", encoding="utf-8") as f:
    for adverb in move_adv.text:
        other = list(move_adv.text)
        other.remove(adverb)
        for k, sentence in enumerate(split_rsl):
            if adverb in sentence:
                for other_adverb in other:
                    rus_new = re.sub(adverb, other_adverb, rus[k], flags=re.I)
                    f.write(" ".join([other_adverb if word == adverb else word for word in sentence]) + "\t" +\
                            rus_new.capitalize() + "\n")

## Замена существительных

Теперь можно попробовать заменять субъекты, я беру именно субъекты, потому что там не надо париться с падежом и вероятность того, что предложение будет нормальным выше. Париться надо будет по поводу одушевленности и числа. А еще проверять, что там одно слово, а не два.

In [ ]:
words = pd.read_csv("animation_pos.csv", encoding="utf-8")

In [ ]:
words.tail()

Положу в txt список одуш\неодуш существительных и отсмотрю руками их, чтобы удалить явные ошибки и вхождения, где было несколько слов через запятую (их не стоит разделять на строки, потому что мало ли че там случилось в новой версии словаря).

In [ ]:
act_column = []
for i in range(len(words)):
    if words['pos'][i] == 'S' or words['pos'][i] == 'SPRO':
        # print(words['analysis'][i])
        preanalysis = json.loads(words['analysis'][i].replace("'", '"'))
        if preanalysis[0]['analysis']:
            ## Надо брать не нулевое слово, а то у которого в gr тег S !!
            if 'неод' in preanalysis[0]['analysis'][0]['gr']:
                act_column.append('inan')
            else:
                act_column.append('anim')
        else:
             act_column.append('inan')
    else:
        act_column.append(None)

In [ ]:
len(act_column) == len(words)

In [ ]:
words['animacy'] = act_column

In [ ]:
words[words['animacy'] == 'anim']['text']

In [ ]:
#with open("animate.txt", "w", encoding="utf-8") as f:
#    for word in words[words['animacy'] == 'anim']['text']:
#        f.write(word + "\n")

In [ ]:
#with open("inanimate.txt", "w", encoding="utf-8") as f:
#    for word in words[words['animacy'] == 'inan']['text']:
#        f.write(word + "\n")

Отсмотрела глазами оба файла и почистила их. Теперь их можно заново подгрузить обновленными. А предыдущий код больше не запускать.

In [ ]:
with open("animate.txt", "r", encoding="utf-8") as f:
    anim_nouns = [line.strip('\n') for line in f.readlines()]

In [ ]:
anim_nouns[:4]

Теперь можно загрузить разметки и начать там всё заменять. Но! загрузка разметок у нас с Сашей должна быть одинаковая, поэтому сначала я подгружу в свою папочку его код и сделаю на нем апдейты (мне не нужны никакие теги и лемматизация, только загрузка сырых данных). Я хочу брать именно Сашин код на данном этапе, потому что в моем не хватало второго человечка и второй руки, а у него всё красиво по классам

In [ ]:
rsl_tokens = [sent.split("%>%") for sent in rsl]

Надо отобрать предложения с активным субъектом:

In [ ]:
from pymystem3 import Mystem


class TextStemmer(object):

    def __init__(self):
        self.stemmer = Mystem(disambiguation=False, entire_input=False)
        self.relex = re.compile(r'[,()=|]')
        self.retext = re.compile(r'[^-а-яa-z0-9_]', re.I)

    def _stem_gram(self, text):
        text = self.retext.sub(' ', text)

        def f(lem):
            if lem['analysis']:
                lem = lem['analysis'][0]
                lex = lem['lex']
                gr = self.relex.sub(' ', lem['gr']).split()
            else:
                lex = lem['text']
                gr = []
            return {'lex': lex, 'gram': list(set(gr))}

        text = self.stemmer.analyze(text)
        text = map(f, filter(lambda x: 'analysis' in x, text))

        return list(text)

    def _stem_simple(self, text):
        text = self.retext.sub(' ', text)
        text = self.stemmer.lemmatize(text)
        return [{'lex': t, 'gram': []} for t in text]

    def stem(self, text, gram=True):
        if isinstance(text, str):
            text = [text]

        f = self._stem_simple if not gram else self._stem_gram
        text = [f(t) for t in text]
        lex = [[t['lex'] for t in sentence] for sentence in text]
        gr = [[t['gram'] for t in sentence] for sentence in text]
        return lex, gr

    def __delete__(self, instance):
        self.stemmer.close()

In [ ]:
st = TextStemmer()

stem_sentences_rus, gram_sentences_rus = st.stem(rus[11], gram=True)

print(stem_sentences_rus, gram_sentences_rus, rus[11])

In [ ]:
# это интересно, но лучше собирать всё в широкий датафрейм,
# чтобы не потерять перевод на ржя
selected_sents = defaultdict(list)
for k, sentence in enumerate(rsl_tokens):
    for noun in anim_nouns:
        if noun in sentence:
            selected_sents[noun].append(rus[k])

### Выделяем подлежащее

Теперь мне в selected_sents надо выделить группы подлежащего через udpipe или mystem. А потом проверить, что выделенное подлежащее имеет в себе это самое существительное в именительном падеже.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install ufal.udpipe

In [ ]:
!pip install conllu

In [ ]:
import ufal.udpipe
from model import Model
import conllu
from nltk.parse import DependencyGraph
import networkx as nx

In [ ]:
model = Model('russian-syntagrus-ud-2.4-190531.udpipe')

In [ ]:
def get_conllu(model, text):
    sentences = model.tokenize(text)
    for s in sentences:
        model.tag(s)
        model.parse(s)
    conllu_text = model.write(sentences, "conllu")
    return conllu_text

def get_dep_tree(text):
    trees = []
    for sent in text.split('\n\n'):
        tree = [line for line in sent.split('\n') if line and line[0] != '#']
        trees.append('\n'.join(tree))
    return trees

def get_subtree(nodes, node):
    if not nodes[node]['deps']:
        return [node]
    
    else:
        return [node] + [get_subtree(nodes, dep) for rel in nodes[node]['deps'] 
                         if rel != 'punct'  # пунктуацию доставать не будем
                         for dep in nodes[node]['deps'][rel]]

def flatten(l):
    flat = []
    for el in l:
        if not isinstance(el, list):
            flat.append(el)
        else:
            flat += flatten(el)
    return flat


In [ ]:
selected_sents['человек']

In [ ]:

def extract_subject(selected_sents, noun):
    sents = defaultdict(list)
    for sent in selected_sents[noun]:
        print('=================================')
        print("FULL SENTENCE   ", sent)
        a = get_conllu(model, sent)
        b = get_dep_tree(a)
        d = DependencyGraph(b[0])
        d.root = d.nodes[0]
        #print(list(d.triples()))
        for el in list(d.triples()):
            if el[1] == 'nsubj':
                nsubj = el[2][0]
                #print("NSUBJ    ", nsubj)
                d.nx_graph()
                if nsubj in d.nx_labels.values():
                    ind_subj = list(d.nx_labels.values()).index(nsubj) + 1
                    subject = " ".join([d.nodes[i]['word'] for i in sorted(flatten(get_subtree(d.nodes, ind_subj)))])
                    print("SUBJECT   ", subject)
                    sents[sent].append(subject)
    return sents

def save_sentences_and_subjects(sentences, noun):
    with open(noun + ".csv", "w", encoding="utf-8") as f:
        

In [ ]:
noun = 'человек'
extract_subject(selected_sents, noun)